In [1]:
import pandas as pd
from talib import abstract
import numpy as np

In [2]:

stock = "AMD"

df = pd.read_csv(f"{stock}.csv", parse_dates=True, index_col=0)
df.columns = ["open", "high", "low", "close", "adj close", "volume"]
df.head()

,open,high,low,close,adj close,volume
0,0.386161,0.400670,0.366071,0.372768,0.322123,691101600
1,0.359375,0.375000,0.341518,0.350446,0.302834,1198422400
2,0.362723,0.371652,0.348214,0.357143,0.308621,1191064000
3,0.361607,0.395089,0.357143,0.393973,0.340448,1247752800
4,0.398438,0.415179,0.381696,0.383929,0.331768,820176000


In [3]:
df.interpolate(inplace=True)

In [4]:
df["SMA"] = abstract.SMA(df, timeperiod=7)
df["WMA"] = abstract.WMA(df, timeperiod=7)

In [5]:
df["RSI"] = abstract.RSI(df, timeperiod=7)

In [6]:
df["CCI"] = abstract.CCI(df, timeperiod=10)
df["AD"] = abstract.AD(df, timeperiod=10)

In [7]:
df["STOCHRSIk"] = abstract.STOCHRSI(df, timeperiod=7)["fastk"]
df["STOCHRSId"] = abstract.STOCHRSI(df, timeperiod=7)["fastd"]
df["WILLR"] = abstract.WILLR(df, timeperiod=7)

In [8]:
df["MACD"] = abstract.MACDFIX(df)["macd"]
df["MACDsignal"] = abstract.MACDFIX(df)["macdsignal"]

In [9]:
df = df.iloc[33:]
df.head(50)

,open,high,low,close,adj close,volume,SMA,WMA,RSI,CCI,AD,STOCHRSIk,STOCHRSId,WILLR,MACD,MACDsignal
33,0.354911,0.356027,0.330357,0.333705,0.288368,258792800,0.336097,0.336256,41.424773,-28.818288,-2.925372e+09,42.082633,5.713203e+01,-74.075795,-0.004290,-0.000084
34,0.333705,0.339286,0.320313,0.321987,0.278241,269124800,0.333626,0.332729,34.736127,-132.149818,-3.147007e+09,0.000000,4.736088e+01,-95.312763,-0.005702,-0.001207
35,0.323103,0.327009,0.308036,0.313616,0.271008,492150400,0.331234,0.327726,30.616113,-186.334990,-3.349672e+09,0.000000,1.402754e+01,-88.372820,-0.007376,-0.002441
36,0.297991,0.303571,0.287946,0.294643,0.254613,809597600,0.325016,0.318579,23.306430,-219.654668,-3.465270e+09,0.000000,-9.473903e-15,-90.163188,-0.010032,-0.003959
37,0.303571,0.312500,0.300223,0.304688,0.263292,385705600,0.320552,0.313497,33.163007,-111.110532,-3.570422e+09,54.401096,1.813370e+01,-75.408704,-0.011254,-0.005418
38,0.306920,0.306920,0.293527,0.297991,0.257506,371520800,0.315928,0.307856,30.149171,-103.570827,-3.694281e+09,59.868082,3.808973e+01,-85.245516,-0.012590,-0.006852
39,0.302455,0.311384,0.292411,0.303571,0.262328,613978400,0.310029,0.304767,35.819035,-72.417875,-3.585970e+09,100.000000,7.142306e+01,-77.049397,-0.013081,-0.008098
40,0.261161,0.267857,0.250000,0.255580,0.220857,1374464000,0.298868,0.291155,19.740802,-181.914657,-4.101442e+09,0.000000,5.328936e+01,-93.750420,-0.016919,-0.009862
41,0.257813,0.265625,0.250000,0.255580,0.220857,408917600,0.289381,0.280333,19.740802,-143.533199,-4.218294e+09,0.000000,3.333333e+01,-92.754094,-0.019746,-0.011839
42,0.264509,0.273438,0.257813,0.268973,0.232430,435624000,0.283004,0.275231,31.431967,-90.923409,-4.031638e+09,72.714238,2.423808e+01,-69.643200,-0.020742,-0.013620


# ABOVE USED IN PAPER 

In [10]:
#df["ROC4"] = abstract.ROCP(df, timeperiod=4) * 100

In [11]:
#df["TRIX"] = abstract.TRIX(df)

In [12]:
# Select trend from SMA14 and SMA 50
def MA_trend(ma, price):
    return 1 if price > ma else -1

df["SMA_trend"] = df.apply(lambda x: MA_trend(x["SMA"], x["close"]), axis=1)
df["WMA_trend"] = df.apply(lambda x: MA_trend(x["WMA"], x["close"]), axis=1)

<ipython-input-12-b25718390c1c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SMA_trend"] = df.apply(lambda x: MA_trend(x["SMA"], x["close"]), axis=1)
<ipython-input-12-b25718390c1c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["WMA_trend"] = df.apply(lambda x: MA_trend(x["WMA"], x["close"]), axis=1)


In [13]:
# Select trend from RSI
RSI_trend = [0] * len(df)
for i in range(len(df)):
    if i == 0:
        continue
    lastrsi = df.loc[df.index[i-1], "RSI"]
    rsi = df.loc[df.index[i], "RSI"]
    if rsi > 70:
        RSI_trend[i] = -1
    elif rsi < 30:
        RSI_trend[i] = 1
    elif rsi > lastrsi:
        RSI_trend[i] = 1
    else:
        RSI_trend[i] = -1


df["RSI_trend"] = RSI_trend

<ipython-input-13-bd738f0b9da3>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["RSI_trend"] = RSI_trend


In [14]:
# Select buy trend from STOCHRSI
STOCHRSIk_trend = [0] * len(df)
STOCHRSId_trend = [0] * len(df)
WILLR_trend = [0] * len(df)
CCI_trend = [0] * len(df)
AD_trend = [0] * len(df)
for i in range(len(df)) :
    if i == 0:
        continue
    lastk = df.loc[df.index[i-1], "STOCHRSIk"]
    lastd = df.loc[df.index[i-1], "STOCHRSId"]
    lastwillr = df.loc[df.index[i-1], "WILLR"]
    lastcci = df.loc[df.index[i-1], "CCI"]
    lastad = df.loc[df.index[i-1], "AD"]
    
    k = df.loc[df.index[i], "STOCHRSIk"]
    d = df.loc[df.index[i], "STOCHRSId"]
    willr = df.loc[df.index[i], "WILLR"]
    cci = df.loc[df.index[i], "CCI"]
    ad = df.loc[df.index[i], "AD"]
    # identify trends
    # K
    if k > lastk:
        STOCHRSIk_trend[i] = 1
    else:
        STOCHRSIk_trend[i] = -1
    # D
    if d > lastd:
        STOCHRSId_trend = 1
    else:
        STOCHRSId_trend = -1
    # WILLR
    if willr > lastwillr:
        WILLR_trend = 1
    else:
        WILLR_trend = -1
    # CCI
    if cci > 150:
        CCI_trend = -1
    elif cci < -150:
        CCI_trend = 1
    elif cci > lastcci:
        CCI_trend = 1
    else:
        CCI_trend = -1
    # WILLR
    if ad > lastad:
        AD_trend = 1
    else:
        AD_trend = -1
        
        
df["STOCHRSIk_trend"] = STOCHRSIk_trend
df["STOCHRSId_trend"] = STOCHRSId_trend
df["WILLR_trend"] = WILLR_trend
df["CCI_trend"] = CCI_trend
df["AD_trend"] = AD_trend

<ipython-input-14-a364b23c8fc3>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["STOCHRSIk_trend"] = STOCHRSIk_trend
<ipython-input-14-a364b23c8fc3>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["STOCHRSId_trend"] = STOCHRSId_trend
<ipython-input-14-a364b23c8fc3>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [15]:
# Select trend from MACD
def MACD_trend(macd, signal):
    if macd > signal:
        return 1
    else:
        return -1

df["MACD_trend"] = df.apply(lambda x: MACD_trend(x["MACD"], x["MACDsignal"]), axis=1)

<ipython-input-15-da4b33cb6bde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MACD_trend"] = df.apply(lambda x: MACD_trend(x["MACD"], x["MACDsignal"]), axis=1)


In [16]:
# Select trend from TRIX
def TRIX_trend(trix):
    if trix > 0:
        return 1
    else:
        return -1

#df["TRIX_trend"] = df.apply(lambda x: TRIX_trend(x["TRIX"]), axis=1)

In [17]:
df.columns

Index(['open', 'high', 'low', 'close', 'adj close', 'volume', 'SMA', 'WMA',
       'RSI', 'CCI', 'AD', 'STOCHRSIk', 'STOCHRSId', 'WILLR', 'MACD',
       'MACDsignal', 'SMA_trend', 'WMA_trend', 'RSI_trend', 'STOCHRSIk_trend',
       'STOCHRSId_trend', 'WILLR_trend', 'CCI_trend', 'AD_trend',
       'MACD_trend'],
      dtype='object')

In [18]:
df = df.iloc[1:]
outputdf = df.drop(["open", "high", "low", "volume", "adj close",
         'SMA', 'WMA', 'RSI', 'CCI', 'AD',
         'STOCHRSIk', 'STOCHRSId',
         'WILLR', 'MACD', 'MACDsignal'], axis=1)

In [19]:
outputdf.isnull().values.any()

False

In [20]:
len(outputdf)

4745

In [21]:
outputdf.head()

,close,SMA_trend,WMA_trend,RSI_trend,STOCHRSIk_trend,STOCHRSId_trend,WILLR_trend,CCI_trend,AD_trend,MACD_trend
34,0.321987,-1,-1,-1,-1,-1,1,1,1,-1
35,0.313616,-1,-1,-1,-1,-1,1,1,1,-1
36,0.294643,-1,-1,1,-1,-1,1,1,1,-1
37,0.304688,-1,-1,1,1,-1,1,1,1,-1
38,0.297991,-1,-1,-1,1,-1,1,1,1,-1


In [22]:
outputdf.to_csv(f"{stock}trends2.csv", index=False)

In [23]:
# Use 70% for training, and the rest for validation
perc70 = int(len(outputdf) * 0.7)
training_df = outputdf.iloc[:perc70]
validation_df = outputdf.iloc[perc70:]

In [24]:
training_df.to_csv(f"{stock}training.csv", index=False)
validation_df.to_csv(f"{stock}validation.csv", index=False)